In [1]:
import numpy as np
import time, sys 
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle
from sklearn.svm import LinearSVC,SVC
from sklearn.model_selection import KFold
from scipy import signal
# import self defined functions 
from riemannian_multiscale import riemannian_multiscale
from filters import load_filterbank 
from get_data import get_data
from sklearn.preprocessing import LabelBinarizer
from csp import generate_projection,generate_eye,extract_feature
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split


In [38]:
def bandpass(start,stop,data,fs):
    bp_Hz = np.array([start, stop])
    b, a = signal.butter(5, bp_Hz / (fs / 2.0), btype='bandpass')
    return signal.lfilter(b, a, data, axis=0)

def notch(val, data, fs):
    bp_stop_Hz = val + 3.0 * np.array([-1, 1])
    b, a = signal.butter(3, bp_stop_Hz / (fs / 2.0), 'bandstop')
    return signal.lfilter(b, a, data)

def data_preposcessing(x,y,chunk,fs,band,nothval,samplesize,channel_num=8):
    #Divide datas to chunks, each chunk coorespond to specific channel with specific label.
    #Total chunk number = channel_num * record_size(chunk) *3
    _total_chunk=channel_num*chunk*3

    for k in range(channel_num):
        locals()["channel"+str(k+1)]=x[:,k]

    eventsplit=[]
    for u in range(int(chunk*3)):
        event_locator=u*samplesize
        eventsplit.append(int(event_locator))

    for i in np.unique(y):
        i=int(i)
        locals()['ind'+str(i+1)]=[]
    for split in eventsplit:
        label_finder=int(y[split])
        locals()['ind'+str(label_finder+1)].append([split,int(split+samplesize-1)])
    for i in np.unique(y):
        i=int(i)
    #     locals()['ind_'+str(i+1)]=np.where(y==i)[0] ##left=-1,(ind0) ; right=1,(ind2); focus=0,(ind1);

    #     locals()['ind'+str(i+1)]=[locals()['ind_'+str(int(i+1))][0]]  
    #     for j in range(len(locals()['ind_'+str(i+1)])-1):  
    #         if (locals()['ind_'+str(i+1)])[j]+1!=(locals()['ind_'+str(i+1)])[j+1]:
    #             locals()['ind'+str(i+1)].append((locals()['ind_'+str(i+1)])[j])
    #         if j>1 and (locals()['ind_'+str(i+1)])[j]-1!=(locals()['ind_'+str(i+1)])[j-1]:
    #             (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[j])
    #     (locals()['ind'+str(i+1)]).append((locals()['ind_'+str(i+1)])[-1])   #find boundry of events

        if chunk!=len((locals()['ind'+str(i+1)])):
            print("error: chunk size and interval limit does not match. flag number:",i)
        for k in range(channel_num):
            for g in range(int(len((locals()['ind'+str(i+1)])))):
                locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]=locals()['channel'+str(k+1)][(locals()['ind'+str(i+1)])[g][0]:(locals()['ind'+str(i+1)])[g][1]+1]

    total_chunks=0
    #apply bp and notch filter
    #variable_channel#_label#_chunk#

    for i in np.unique(y):
        i=int(i)
        for k in range(channel_num):
            for g in range(int(len((locals()['ind'+str(i+1)])))):
                data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
                bp_filtered=bandpass(band[0],band[1],data,fs) #apply band pass filter
                channel_filtered=notch(notchval,bp_filtered,fs)
                locals()['bp_filtered'+str(k+1)+'_'+str(i+1)+'_'+str(g+1)]=channel_filtered
                total_chunks+=1
    if total_chunks != _total_chunk: print('error: total chunks number does not match with the data acqusition')
    ##################CNN input matrix generation#################################
    labels=[]
    for i in np.unique(y): 
        i=int(i)
        for g in range(int(len((locals()['ind'+str(i+1)])))):
            for k in range(channel_num):
                data=locals()['channel'+str(k+1)+'_'+str(i+1)+'_chunk'+str(g+1)]
                if k==0:
                    CNNmatrices_mergechn=[data]
                else:
                    CNNmatrices_mergechn=np.append(CNNmatrices_mergechn,[data],axis=0)
            if g==0 and i==-1: CNNmatrices=CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1])
            else: CNNmatrices=np.append(CNNmatrices,CNNmatrices_mergechn.reshape(1,CNNmatrices_mergechn.shape[0],CNNmatrices_mergechn.shape[1]),axis=0)
            labels=np.append(labels,[i])
    #     _shape=CNNmatrices.shape
    #     CNN_input=CNNmatrices.reshape(_shape[0],1,_shape[1],_shape[2])
    lb = LabelBinarizer()
    label_enc=lb.fit_transform(labels)
    return CNNmatrices,labels

In [52]:
# set path to datasets
datapath = '/home/jingyan/Documents/ME499-WinterProject/mindwave/src/motor_cortex_ml/data/'
for run in range(1,7): #6runs for each session
    savetag='_exp10_sec1_run'+str(run)
    csvname =datapath + 'record'+savetag+'.csv'
    data = np.loadtxt(csvname,delimiter = ',')
    if run==1:
        x = data[:,:-1]
        y = data[:,-1]
    else:
        x=np.append(x,data[:,:-1],axis=0)
        y=np.append(y,data[:,-1],axis=0)
channel_num=8
chunk=60
fs=250
band=(0.5,100)
notchval=60
samplesize=7.5*250

train_data,train_labels=data_preposcessing(x,y,chunk,fs,band,notchval,samplesize)

In [40]:
#keep two classes, left and right
remove_list=[0]
drop_list=np.isin(train_labels,remove_list,invert=True)
drop_array=np.array(drop_list)
label_keep=train_labels[drop_array]
train_keep=train_data[drop_array]
# drop_list2=np.isin(eval_label,remove_list,invert=True)
# drop_array2=np.array(drop_list2)
# eval_keep=eval_sample[drop_array2]
# eval_label_keep=eval_label[drop_array2]

# Riemann Approach

In [64]:
bw=np.array([2,4,8,16,32])
forder=2
filter_bank=load_filterbank(bw,fs,order=forder,max_freq=40)
time_windows_flt = np.array([[2.5,4.5],
                            [4,6],
                            [2.5,6],
                            [2.5,3.5],
                            [3,4],
                            [4,5]])*fs
time_windows=time_windows_flt.astype(int)
# time_windows=time_windows[2:3]
rho=0.1

In [65]:
riemann=riemannian_multiscale(filter_bank,time_windows,riem_opt='Riemann',rho=rho,vectorized=True)
train_feat=riemann.fit(train_data) 

###chose eval dataset
# test_feat=riemann.features(eval_data)
x_train,x_test,y_train,y_test=train_test_split(train_feat,train_labels,test_size=0.25)
print(x_train.shape,y_train.shape)

(135, 9288) (135,)


In [66]:
#Linear SVM classifier
clf1 = LinearSVC(C = 0.1, intercept_scaling=1, loss='hinge', max_iter=1000,multi_class='ovr', penalty='l2', random_state=1, tol=0.00001)
clf1.fit(x_train,y_train)
score_train1=clf1.score(x_train,y_train)
score_test1=clf1.score(x_test,y_test)
print(score_train1,score_test1)

1.0 0.7777777777777778


/home/jingyan/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [67]:
#rbf SVM classifier
clf2 = SVC(20,'rbf', degree=10, gamma='auto', coef0=0.0, tol=0.001, cache_size=10000, max_iter=-1, decision_function_shape='ovr')
clf2.fit(x_train,y_train)
score_train2=clf2.score(x_train,y_train)
score_test2=clf2.score(x_test,y_test)
print(score_train2,score_test2)

1.0 0.6444444444444445


In [68]:
# LDA classifier
clf3=LinearDiscriminantAnalysis()
clf3.fit(x_train,y_train)
score_train3=clf3.score(x_train,y_train)
score_test3=clf3.score(x_test,y_test)
print(score_train3,score_test3)

0.7703703703703704 0.6666666666666666


/home/jingyan/.local/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


# CSP Approach

In [75]:
bw=np.array([2,4,8,16,32])
forder=2
filter_bank=load_filterbank(bw,fs,order=forder,max_freq=40)
time_windows_flt = np.array([
                            [2.5,3.5],
                            [3,4],
                            [3.5,4.5],
                            [4,5],
                            [4.5,5.5],
                            [5,6],
                            [2.5,4.5],
                            [3,5],
                            [3.5,5.5],
                            [4,6],
                            [2.5,6]])*fs
time_windows = time_windows_flt.astype(int)
NO_csp=24


In [77]:
w = generate_projection(train_data,train_labels,NO_csp,filter_bank,time_windows)
# feature_mat = extract_feature(train_data,w,filter_bank,time_windows)

/home/jingyan/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jingyan/.local/lib/python3.6/site-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


ValueError: array must not contain infs or NaNs